In [152]:
import json 
import re
from bs4 import BeautifulSoup
import requests

relevant_words = {}
with open("Resources/SiteInfo.json", "r") as f:
    json_dev_file = json.load(f)

def get_html(URL): #gets html in text
    header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
    page = requests.get(URL,headers=header) 
    soup = BeautifulSoup(page.text,'html.parser') #take html
    return soup

##going with simplicity here, 
def grab_hrefs(page_elem):
    tags = []
    tags = page_elem.find_all('a')
    links = [tag['href'] for tag in tags if 'href' in tag.attrs] #find links
    return links

def grab_id_parent(page_elem,pg_id):
    return page_elem.find(id=pg_id)
def grab_scripts(page_elem,regex = re.compile('')):
    #return [script for script in page_elem.find_all("script") if regex.search(script.attrs['src'])]
    return page_elem.find_all("script")
    
#IN USE NOW 
def grab_team_hrefs(page_elem):
    links = grab_hrefs(page_elem)

    ####REWORK vvvvv to work with other states via an container
    team_links = [link for link in links if link.startswith("/ca/") and link.endswith("/basketball/") and link != "/ca/basketball/"]
    ####HARDCODED RN but can use relevant words from parse_url_context in the future to decipher relevant links
    return team_links

def grab_player_hrefs(page_elem): #eventually should make respective classes that override this function
    links = grab_hrefs(page_elem)
    relevant_pattern = r'careerid=[a-z0-9]+' #eventually, it should read off json to get a regex depending on site
    player_links = [link for link in links if re.search(relevant_pattern,link)]
    return player_links
#WIP

def grab_player_info(page_elem):
    ##there is a big script.
    player_scripts = grab_id_parent(ex_player_pg, '__NEXT_DATA__')
    return

def get_value_from_key_list(json_file,list_of_keys):
    current_lvl = json_file
    #add error handling in future
    for key in list_of_keys:
        current_lvl = current_lvl[key]
    return current_lvl


In [120]:
laf_ball_roster_html = get_html("https://www.maxpreps.com/ca/lafayette/acalanes-dons/basketball/roster/")

In [121]:
team_page_html = get_html("https://www.maxpreps.com/ca/basketball/schools/") 

In [122]:
team_links = grab_team_hrefs(team_page_html)
print(len(team_links)) #200 is number of teams

200


In [123]:
#FOR SOME REASON THEY HAVE A SPACE which throws off search.
#develop context based title finding 
member_links = grab_roster_hrefs(laf_ball_roster_html)
print(f'{len(member_links)}') #as of rn, there are 13 players on this team

13


In [124]:
site_url = json_dev_file['maxpreps']['site_url']
ex_player_pg = get_html(f"{site_url}{member_links[-1]}")

In [125]:
player_json_script = grab_id_parent(ex_player_pg, '__NEXT_DATA__')
json_data = json.loads(player_json_script.string)
with open('ex_player_info.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, indent=4)


In [153]:
with open('ex_player_info.json', 'r') as f:
    ex_player_file = json.load(f)
data_map = json_dev_file['maxpreps']['data_structure']['key_to_key_maps']
data_map_keys = list(data_map.keys())
general_info_path = data_map[data_map_keys[0]]
general_info = get_value_from_key_list(ex_player_file,general_info_path)

KeyError: 'props '

In [126]:
##drop irrelevant keys on json file  
##categorize

In [127]:
redundant_words = {"maxpreps","http","https","www","com"} #could also be relevant words when looking for links that match page context 
URL = "https://www.maxpreps.com/ca/basketball/schools/"
url_words = [w for w in re.split(r'[/\.:]+',URL) if w.lower() not in redundant_words and len(w) > 0]
url_words

['ca', 'basketball', 'schools']

In [128]:
target_text = soup.find_all(['h1','h2','h3'])
target_text[-1].get_text()
target_text #WIP on pinpointing exact div-- for the far future

NameError: name 'soup' is not defined

In [ ]:
def parse_url_context(URL, redundant_words): # remove maxpreps and https:// etc but keep keywords like ca
    # sc, Basketball, Boys to use and find relevant themes. This way, we can remove redundant words from links within page that may point to itself
    #can use these words and additional context to generate likelihood of teams div
    #not use right now
    #chop up url
    url_words = [w for w in re.split(r'[/\.:]+',URL) if w.lower() not in redundant_words and len(w) > 0]
    return url_words
